# 2nd level group statistics to test
1. Stimulation effect (one sample t-test of 1st level results)
2. Dopamine amplitude effect on evoked map (one-way anova)
3. Stimulation applitude effect on evoked map (one-way anova)

In [8]:
# import packages we will need
%pylab inline
import pickle
import hrf_estimation as he
import seaborn as sns
from fmri_stats import *

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge

# data folder
data_pm = PathMan('./Data')      # put all data in here
iput_pm = PathMan('./Results')   # the results will be generated here
cach_pm = PathMan('./Caches')    # all intermediate files will be saved here
oput_pm = PathMan('./Stats')

Populating the interactive namespace from numpy and matplotlib


### Load Peak DA values (received this from Lindsay)

In [9]:
import pandas as pd

meta_df = pd.read_excel(data_pm('./DA_MaxValues.xlsx'))
subj_map = dict(s1='19', s2='20', s3='21', 
                s4='24', s5='25', s6='26')
stim_map = dict(three='3', five='5', seven='7')

max_da = dict()
for i, row in meta_df.iterrows():
    subj = subj_map[row['Subj']]
    stim = stim_map[row['StimAmp']]
    run = row['StimOrder']
    da = row['DA']
    did = f'{subj}-{stim.zfill(2)}-{str(run).zfill(2)}'
    max_da[did] = da
del meta_df

### One sample T-test (compare canonical HRF and homemade HRF)

In [10]:
# set p value. The p value here will apply after multiple comparison correction
pval = 0.05

### Load 1st stats results of Estimated HRF

In [11]:
import nibabel as nib

msk_nii = nib.load(data_pm('Rat_BrainMask-4slices.nii.gz'))
msk_idx = np.nonzero(msk_nii.dataobj)

num_voxs = msk_idx[0].shape[0]
num_subj = len(max_da)

dict_keyset = ['SubjectID', 'StimAmp', 'DopaAmp', 'Trial']
dict_meta = {c:[] for c in dict_keyset}

# This pattern identify the evoked response map of each trial estimated on 1st level analysis
# load evoked response map 'stats_XX-XX-XX_est.nii.gz'
fname_ptrn = r'stats_(?P<subj>[0-9]{2})-(?P<stim>[0-9]{2})-(?P<run>[0-9]{2})_est\.nii\.gz'
data_array_est = np.zeros([num_subj, num_voxs])

j = 0
for i, fname in iput_pm.listdir(pattern=fname_ptrn).items():
    mtch = re.match(fname_ptrn, fname)
    if mtch is not None:
        subj = mtch.group('subj')
        stim = mtch.group('stim')
        run = mtch.group('run')
        
        did = f'{subj}-{stim}-{run}'
        maxd = max_da[did]

        dict_meta['SubjectID'].append(subj)
        dict_meta['StimAmp'].append(stim)
        dict_meta['DopaAmp'].append(maxd)
        dict_meta['Trial'].append(run)

        # standardize response map generated with Dopamine HRF
        data = np.asarray(nib.load(iput_pm(fname)).dataobj)[msk_idx]
        data_z = (data.copy() - data.mean()) / data.std()
        data[abs(data_z) < stats.norm.ppf(1-pval)] = 0
        data_array_est[j, :] = data
        j += 1
        
# The meta information contains DA amplitude, Stim amplitude, subject ID, Trials number
meta_df = pd.DataFrame(dict_meta)

In [12]:
meta_df

,SubjectID,StimAmp,DopaAmp,Trial
0,24,05,85.2,02
1,20,03,38.0,01
2,21,03,37.9,03
3,19,03,9.4,02
4,20,07,88.0,01
5,19,07,22.8,02
6,21,07,89.2,03
7,24,07,99.3,01
8,25,07,44.5,03
9,24,03,44.6,01


In [6]:
# the permutation-based one sample t-test will be applied, 
# the multiple comparison was adopted through Max T method

stats_1samp_img = np.zeros(msk_nii.shape)
stats_1samp_dat, t_orig, p_orig = onesample_ttest(data_array_est, 
                                                  pval=pval, nperm=5000, twosided=True)
stats_1samp_dat[p_orig > pval] = 0
stats_1samp_img[msk_idx] = stats_1samp_dat
stats_1samp_nii = nib.Nifti1Image(stats_1samp_img, msk_nii.affine)
stats_1samp_nii.header['qform_code'] = 1
stats_1samp_nii.header['sform_code'] = 0
stats_1samp_nii.to_filename(oput_pm('stats_1samp_est.nii.gz'))

## One-way ANOVA
- We used one-way ANOVA to test which factor (Dopamine amplitude from FSCV vs Current amplitude of DBS stimulation). 

In [7]:
# Below code will perform permutation-based anova with each factor, n_permute = 5000
F_s, P_s = anova1_lm_permute('StimAmp', meta_df, data_array_est)
F_d, P_d = anova1_lm_permute('DopaAmp', meta_df, data_array_est)

# Below code will be used to determine critical F to threshold final map.
_, p_eff_s = anova1_lm_multi('StimAmp', meta_df, data_array_est)
_, p_eff_d = anova1_lm_multi('DopaAmp', meta_df, data_array_est)

# The first threshold will apply for group level significant. Its not related to
# the effect size of responses, but only group level significant.
da_effect = F_d[0]
da_effect[abs(P_d[0]) > 0.05] = 0
# by applying below threshold, we remove all non-relevant results (low F values)
critical_F_d = F_d[0][p_eff_d[0] < pval].min()
da_effect[da_effect < critical_F_d] = 0

# save to file
da_effect_img = np.zeros(msk_nii.shape)
da_effect_img[msk_idx] = da_effect
de_nii = nib.Nifti1Image(da_effect_img, msk_nii.affine, msk_nii.header)
de_nii.to_filename(oput_pm('DaEffect_oneway_est.nii.gz'))


# do the same for Stimulation amp effect
stim_effect = F_s[0]
stim_effect[abs(P_s[0]) > 0.05] = 0
critical_F_s = F_s[0][p_eff_s[0] < pval].min()
stim_effect[stim_effect < critical_F_s] = 0

stim_effect_img = np.zeros(msk_nii.shape)
stim_effect_img[msk_idx] = stim_effect

se_nii = nib.Nifti1Image(stim_effect_img, msk_nii.affine, msk_nii.header)
se_nii.to_filename(oput_pm('StimEffect_oneway_est.nii.gz'))

/Users/sasha/201029_LW-FSCV_AnalysisPackage/fmri_stats.py:127: RuntimeWarning: invalid value encountered in true_divide
  fvals = mean_sqs/mean_sq_error
/Users/sasha/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


/Users/sasha/201029_LW-FSCV_AnalysisPackage/fmri_stats.py:148: RuntimeWarning: invalid value encountered in greater
  pvals_perm[i, : ] = [(null_dist[i, :] > f).astype(np.int16).sum() / nperm for f in fval]


/Users/sasha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  
/Users/sasha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app
/Users/sasha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in less
/Users/sasha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less
